In [5]:
!wget -nc https://lazyprogrammer.me/course_files/spam.csv

--2020-10-23 00:22:53--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.31.81.48, 172.67.213.166, 104.31.80.48, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.31.81.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.05s   

2020-10-23 00:22:53 (9.24 MB/s) - ‘spam.csv’ saved [503663/503663]



In [92]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [108]:
data = pd.read_csv("spam.csv", encoding = "ISO-8859-1")

In [109]:
data = data[['v1', 'v2']]
data.columns = ['label', 'text']
data['label'] = data['label'].map({'ham': 0, 'spam': 1})
Y = data['label'].values

In [110]:
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [111]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], Y, test_size=0.33, random_state=101)
# print(X_train[0])

In [112]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3733,), (1839,), (3733,), (1839,))

In [113]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)

In [114]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [115]:
X_train = pad_sequences(X_train)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])
X_train.shape, X_test.shape

((3733, 121), (1839, 121))

In [116]:
V = len(tokenizer.word_index)
V

7246

In [122]:
T = X_train.shape[1]
T

121

In [123]:
y_train[81:115]

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [139]:
D =20
M =15
i = Input(shape=(T,))
X = Embedding(V+1, D)(i)
X = LSTM(M, return_sequences=True)(X)
X = GlobalMaxPooling1D()(X)
X = Dense(1, activation='sigmoid')(X)

model = Model(i, X)





In [140]:
# Compile and fit
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


print('Training model...')
r = model.fit(
  X_train,
  y_train,
  epochs=10,
  validation_data=(X_test, y_test)
)

Training model...
Epoch 1/10
117/117 [==============================] - 6s 54ms/step - loss: 0.4651 - accuracy: 0.8572 - val_loss: 0.2927 - val_accuracy: 0.8749
Epoch 2/10
117/117 [==============================] - 6s 48ms/step - loss: 0.2204 - accuracy: 0.9226 - val_loss: 0.1635 - val_accuracy: 0.9739
Epoch 3/10
117/117 [==============================] - 6s 47ms/step - loss: 0.1254 - accuracy: 0.9863 - val_loss: 0.1109 - val_accuracy: 0.9810
Epoch 4/10
117/117 [==============================] - 6s 48ms/step - loss: 0.0791 - accuracy: 0.9912 - val_loss: 0.0828 - val_accuracy: 0.9826
Epoch 5/10
117/117 [==============================] - 6s 48ms/step - loss: 0.0532 - accuracy: 0.9944 - val_loss: 0.0689 - val_accuracy: 0.9853
Epoch 6/10
117/117 [==============================] - 6s 48ms/step - loss: 0.0381 - accuracy: 0.9965 - val_loss: 0.0604 - val_accuracy: 0.9880
Epoch 7/10
117/117 [==============================] - 6s 48ms/step - loss: 0.0288 - accuracy: 0.9971 - val_loss: 0.0543 - va

In [158]:
EmbeddingsMatrix = model.layers[1].get_weights()[0]
# EmbeddingsMatrix

In [159]:
def get_embed(word):
  word_index = tokenizer.word_index[word]
  return EmbeddingsMatrix[word_index]

In [168]:
from scipy import spatial
cosine = spatial.distance.cosine

In [190]:
1 -cosine(get_embed("heart") ,get_embed("doctor"))

0.722819447517395

In [191]:
1 - cosine(get_embed("parents") ,get_embed("mom"))

0.7496672868728638

In [192]:
1 - cosine(get_embed("dad") ,get_embed("man"))

0.8113517761230469

In [196]:
1- cosine(get_embed("dad") - get_embed("man") + get_embed("women"), get_embed("mom"))

0.4567147195339203

In [172]:
tokenizer.word_index

{'i': 1,
 'to': 2,
 'you': 3,
 'a': 4,
 'the': 5,
 'u': 6,
 'and': 7,
 'in': 8,
 'is': 9,
 'me': 10,
 'my': 11,
 'your': 12,
 'for': 13,
 'it': 14,
 'call': 15,
 'of': 16,
 'have': 17,
 '2': 18,
 'on': 19,
 'now': 20,
 'that': 21,
 'are': 22,
 'but': 23,
 'so': 24,
 'not': 25,
 'do': 26,
 'or': 27,
 "i'm": 28,
 'with': 29,
 'at': 30,
 'ur': 31,
 'can': 32,
 'get': 33,
 'be': 34,
 'just': 35,
 'no': 36,
 'if': 37,
 'will': 38,
 'this': 39,
 'we': 40,
 '4': 41,
 'gt': 42,
 'lt': 43,
 'when': 44,
 'up': 45,
 'from': 46,
 'go': 47,
 'how': 48,
 'free': 49,
 'out': 50,
 'all': 51,
 'know': 52,
 'what': 53,
 'ok': 54,
 'good': 55,
 'like': 56,
 'got': 57,
 'am': 58,
 'its': 59,
 'was': 60,
 'then': 61,
 'love': 62,
 'day': 63,
 'come': 64,
 'only': 65,
 'time': 66,
 'want': 67,
 'send': 68,
 'he': 69,
 'txt': 70,
 'as': 71,
 'one': 72,
 'text': 73,
 "i'll": 74,
 'there': 75,
 'about': 76,
 'going': 77,
 'home': 78,
 'stop': 79,
 'need': 80,
 'lor': 81,
 'r': 82,
 'still': 83,
 'n': 84,
 'by'